In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
steepness = 50    # steepness factor
num_blocks = 1000 # number of expression blocks
num_data = 9100   # number of data
test_size = 100   # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# parameters as input data
p_low, p_high = 1.0, 8.0
a_low, a_high = 0.5, 4.5
p_train = np.random.uniform(p_low, p_high, (train_size, 1)).astype(np.float32)
p_test  = np.random.uniform(p_low, p_high, (test_size, 1)).astype(np.float32)
p_dev   = np.random.uniform(p_low, p_high, (val_size, 1)).astype(np.float32)
a_train = np.random.uniform(a_low, a_high, (train_size, num_blocks)).astype(np.float32)
a_test  = np.random.uniform(a_low, a_high, (test_size, num_blocks)).astype(np.float32)
a_dev   = np.random.uniform(a_low, a_high, (val_size, num_blocks)).astype(np.float32)

In [5]:
# nm datasets
from neuromancer.dataset import DictDataset
data_train = DictDataset({"p":p_train, "a":a_train}, name="train")
data_test = DictDataset({"p":p_test, "a":a_test}, name="test")
data_dev = DictDataset({"p":p_dev, "a":a_dev}, name="dev")
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 64
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=True)

## Exact Solver

In [6]:
from src.problem import msRosenbrock
model = msRosenbrock(steepness, num_blocks, timelimit=60)

In [22]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # set params
    model.set_param_val({"p":p, "a":a})
    # solve
    tick = time.time()
    try:
        xval, objval = model.solve("scip")
        # eval
        params.append(list(p)+list(a))
        sols.append(list(list(xval.values())[0].values()))
        objvals.append(objval)
        conviols.append(sum(model.cal_violation()))
    except:
        params.append(list(p)+list(a))
        sols.append(None)
        objvals.append(None)
        conviols.append(None)
    tock = time.time()
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
df.to_csv("result/rb_exact_50-1000.csv")
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
print("Number of None values: ", df["Sol"].isna().sum())

100%|██████████████████████████████████████████████████████████████████████████████| 100/100 [1:40:16<00:00, 60.16s/it]


            Obj Val  Constraints Viol  Elapsed Time
count  1.000000e+02        100.000000    100.000000
mean   1.034866e+10         60.337900     60.154477
std    9.777445e+10        365.341487      0.391713
min    7.003418e+03          0.000000     59.505559
25%    8.094900e+03          0.000000     59.961267
50%    9.235170e+03          0.000000     60.029059
75%    1.113104e+04          0.000000     60.151567
max    9.774853e+11       2798.928200     61.952170
Number of infeasible solution: 3
Number of None values:  0


## Heuristic - Round

In [23]:
from src.heuristic import naive_round

In [24]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # set params
    model.set_param_val({"p":p, "a":a})
    # relax
    model_rel = model.relax()
    # solve
    tick = time.time()
    xval_rel, _ = model_rel.solve("scip")
    xval, objval = naive_round(xval_rel, model)
    tock = time.time()
    # eval
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
df.to_csv("result/rb_heur_rnd_50-1000.csv")
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [35:15<00:00, 21.15s/it]


            Obj Val  Constraints Viol  Elapsed Time
count  1.000000e+02        100.000000    100.000000
mean   8.608484e+06        712.702877     20.863169
std    1.728931e+07       1002.559172     25.682819
min    4.489367e+03          0.000000      2.119940
25%    6.923767e+03          0.000000      2.229135
50%    7.189048e+03         40.970107      2.357334
75%    9.150833e+03       1190.632737     60.004233
max    4.301407e+07       4576.590539     61.145267
Number of infeasible solution: 55


## Heuristic - N1

In [25]:
model_heur = model.first_solution_heuristic(nodes_limit=1)

In [26]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # set params
    model_heur.set_param_val({"p":p, "a":a})
    # solve
    tick = time.time()
    try:
        xval, objval = model_heur.solve("scip")
        # eval
        params.append(list(p)+list(a))
        sols.append(list(list(xval.values())[0].values()))
        objvals.append(objval)
        conviols.append(sum(model_heur.cal_violation()))
    except:
        params.append(list(p)+list(a))
        sols.append(None)
        objvals.append(None)
        conviols.append(None)
    tock = time.time()
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
df.to_csv("result/rb_heur_n1_50-1000.csv")
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [14:36<00:00,  8.76s/it]


            Obj Val  Constraints Viol  Elapsed Time
count  1.000000e+02        100.000000    100.000000
mean   2.098864e+13         60.337900      8.753145
std    1.104401e+14        365.341487      9.631544
min    7.003418e+03          0.000000      5.077394
25%    8.109379e+03          0.000000      5.503791
50%    9.235170e+03          0.000000      6.291873
75%    1.113104e+04          0.000000      7.590513
max    9.017647e+14       2798.928200     60.242697
Number of infeasible solution: 3


## Learnable Rounding

In [7]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [8]:
# hyperparameters
penalty_weight = 100   # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 256           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [9]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmRosenbrock(["p", "a", "x_rnd"], steepness, num_blocks, penalty_weight)

In [10]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 250300.11
Epoch 1, Validation Loss: 105608.96
Epoch 2, Validation Loss: 27104.90
Epoch 3, Validation Loss: 54222.02
Epoch 4, Validation Loss: 37978.01
Epoch 5, Validation Loss: 16898.56
Epoch 6, Validation Loss: 32638.00
Epoch 7, Validation Loss: 58506.22
Epoch 8, Validation Loss: 21326.39
Epoch 9, Validation Loss: 15854.39
Epoch 10, Validation Loss: 9592.15
Epoch 11, Validation Loss: 9759.54
Epoch 12, Validation Loss: 10781.09
Epoch 13, Validation Loss: 10186.48
Epoch 14, Validation Loss: 13282.76
Epoch 15, Validation Loss: 8255.31
Epoch 16, Validation Loss: 9875.97
Epoch 17, Validation Loss: 10855.83
Epoch 18, Validation Loss: 10604.38
Epoch 19, Validation Loss: 11615.07
Epoch 20, Validation Loss: 7679.64
Epoch 21, Validation Loss: 7050.09
Epoch 22, Validation Loss: 14636.49
Epoch 23, Validation Loss: 8134.74
Epoch 24, Validation Loss: 7438.54
Epoch 25, Validation Loss: 7831.09
Epoch 26, Validation Loss: 7250.64
Epoch 27, Validation Loss: 14208.64
Epoch 28, 

In [11]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
df.to_csv("result/rb_lr_50-1000.csv")
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:12<00:00,  8.30it/s]


            Obj Val  Constraints Viol  Elapsed Time
count    100.000000        100.000000    100.000000
mean    6245.146850          0.527028      0.004178
std     1871.580588          3.971328      0.000894
min     2750.472117          0.000000      0.002979
25%     4614.220490          0.000000      0.003531
50%     6500.151225          0.000000      0.004006
75%     7880.428093          0.000000      0.004545
max    10655.660961         36.360297      0.008592
Number of infeasible solution: 2


## Learnable Threshold

In [12]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [13]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 256           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [14]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                          int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmRosenbrock(["p", "a", "x_rnd"], steepness, num_blocks, penalty_weight)

In [15]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 226581.63
Epoch 1, Validation Loss: 75214.19
Epoch 2, Validation Loss: 19010.10
Epoch 3, Validation Loss: 15991.60
Epoch 4, Validation Loss: 14844.77
Epoch 5, Validation Loss: 14964.37
Epoch 6, Validation Loss: 29084.75
Epoch 7, Validation Loss: 34681.92
Epoch 8, Validation Loss: 17475.70
Epoch 9, Validation Loss: 61928.88
Epoch 10, Validation Loss: 42145.68
Epoch 11, Validation Loss: 11557.54
Epoch 12, Validation Loss: 32102.24
Epoch 13, Validation Loss: 22599.27
Epoch 14, Validation Loss: 7574.08
Epoch 15, Validation Loss: 15809.14
Epoch 16, Validation Loss: 22126.03
Epoch 17, Validation Loss: 15885.51
Epoch 18, Validation Loss: 8336.59
Epoch 19, Validation Loss: 8245.94
Epoch 20, Validation Loss: 6787.02
Epoch 21, Validation Loss: 7074.67
Epoch 22, Validation Loss: 8567.50
Epoch 23, Validation Loss: 7188.14
Epoch 24, Validation Loss: 6200.62
Epoch 25, Validation Loss: 12835.92
Epoch 26, Validation Loss: 6362.29
Epoch 27, Validation Loss: 6384.37
Epoch 28, V

In [16]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
df.to_csv("result/rb_lr_50-1000.csv")
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:14<00:00,  6.95it/s]


           Obj Val  Constraints Viol  Elapsed Time
count   100.000000        100.000000    100.000000
mean   6224.790435          1.652397      0.004661
std     651.029131         10.427171      0.001050
min    4860.101613          0.000000      0.002997
25%    5722.318798          0.000000      0.004008
50%    6097.228028          0.000000      0.004496
75%    6679.180990          0.000000      0.005120
max    7619.710902         82.268508      0.008314
Number of infeasible solution: 3


## Parametric Learning Then Rounding

In [17]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [18]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hsize = 256           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [19]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# build neuromancer problem for rounding
components = nn.ModuleList([smap]).to("cuda")
loss_fn = nmRosenbrock(["p", "a", "x"], steepness, num_blocks, penalty_weight)

In [20]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 227071.21
Epoch 1, Validation Loss: 42929.58
Epoch 2, Validation Loss: 40075.25
Epoch 3, Validation Loss: 22170.50
Epoch 4, Validation Loss: 33348.82
Epoch 5, Validation Loss: 22534.24
Epoch 6, Validation Loss: 5636.01
Epoch 7, Validation Loss: 14762.74
Epoch 8, Validation Loss: 16995.27
Epoch 9, Validation Loss: 3911.44
Epoch 10, Validation Loss: 8198.72
Epoch 11, Validation Loss: 5865.49
Epoch 12, Validation Loss: 8530.31
Epoch 13, Validation Loss: 3709.07
Epoch 14, Validation Loss: 3393.37
Epoch 15, Validation Loss: 3601.84
Epoch 16, Validation Loss: 2483.33
Epoch 17, Validation Loss: 3558.98
Epoch 18, Validation Loss: 3478.48
Epoch 19, Validation Loss: 4045.05
Epoch 20, Validation Loss: 2511.78
Epoch 21, Validation Loss: 2832.21
Epoch 22, Validation Loss: 2999.51
Epoch 23, Validation Loss: 4738.38
Epoch 24, Validation Loss: 2322.20
Epoch 25, Validation Loss: 5007.01
Epoch 26, Validation Loss: 3789.77
Epoch 27, Validation Loss: 9458.88
Epoch 28, Validation 

In [21]:
from src.heuristic import naive_round
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x"]
    for i in range(num_blocks*2):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval_rel, _ = model.get_val()
    xval, objval = naive_round(xval_rel, model)
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
df.to_csv("result/rb_pr_50-1000.csv")
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:26<00:00,  3.82it/s]


           Obj Val  Constraints Viol  Elapsed Time
count   100.000000        100.000000    100.000000
mean   6186.159721          7.827350      0.001000
std    1148.805519         20.181886      0.000505
min    4430.873318          0.000000      0.000000
25%    5667.174510          0.000000      0.000998
50%    5787.591529          0.000000      0.001001
75%    6141.942020          0.000000      0.001010
max    9931.358833         73.789575      0.002490
Number of infeasible solution: 17


### STE Rounding

In [7]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [8]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hsize = 16            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [9]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundSTEModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
rnd = roundSTEModel(param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], int_ind=model.int_ind, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmRosenbrock(["p", "a", "x_rnd"], steepness, num_blocks, penalty_weight)

In [10]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 226551.41
Epoch 1, Validation Loss: 55871.21
Epoch 2, Validation Loss: 52922.65
Epoch 3, Validation Loss: 49271.24
Epoch 4, Validation Loss: 48145.26
Epoch 5, Validation Loss: 40413.80
Epoch 6, Validation Loss: 27387.58
Epoch 7, Validation Loss: 24182.88
Epoch 8, Validation Loss: 18316.17
Epoch 9, Validation Loss: 14350.27
Epoch 10, Validation Loss: 14028.93
Epoch 11, Validation Loss: 14611.12
Epoch 12, Validation Loss: 12413.66
Epoch 13, Validation Loss: 25613.50
Epoch 14, Validation Loss: 11987.83
Epoch 15, Validation Loss: 13011.59
Epoch 16, Validation Loss: 10972.10
Epoch 17, Validation Loss: 11717.30
Epoch 18, Validation Loss: 10446.19
Epoch 19, Validation Loss: 13584.45
Epoch 20, Validation Loss: 21774.82
Epoch 21, Validation Loss: 10303.77
Epoch 22, Validation Loss: 9079.90
Epoch 23, Validation Loss: 26099.71
Epoch 24, Validation Loss: 9577.48
Epoch 25, Validation Loss: 11399.56
Epoch 26, Validation Loss: 8562.43
Epoch 27, Validation Loss: 9155.26
Epoch

In [11]:
from src.heuristic import naive_round
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x"]
    for i in range(num_blocks*2):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval_rel, _ = model.get_val()
    xval, objval = naive_round(xval_rel, model)
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
df.to_csv("result/rb_st_50-1000.csv")
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:17<00:00,  5.67it/s]


           Obj Val  Constraints Viol  Elapsed Time
count   100.000000             100.0    100.000000
mean   6566.854725               0.0      0.002689
std     659.557880               0.0      0.001257
min    5496.507491               0.0      0.001005
25%    6108.953148               0.0      0.002013
50%    6441.978779               0.0      0.002582
75%    6771.747828               0.0      0.003006
max    8764.861812               0.0      0.013798
Number of infeasible solution: 0
